<a href="https://colab.research.google.com/github/SouravSahu/GEC/blob/master/GEC__1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install spacy

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

dev_decoder_in.npy   model.png		    train_encoder_in.npy
dev_decoder_out.npy  sample_data	    wi+locness
dev_encoder_in.npy   train_decoder_in.npy   wi+locness_v2.bea19.tar.gz
drive		     train_decoder_out.npy


# **Getting Data**

In [1]:
!python -m spacy download en_core_web_md

    100% |████████████████████████████████| 120.9MB 50.6MB/s 
  Running setup.py install for en-core-web-md ... done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [0]:
import spacy
import os
import numpy as np

In [0]:
nlp=spacy.load("en_core_web_md")

In [4]:
len(nlp.vocab.strings)

1482069

In [5]:
!wget https://www.cl.cam.ac.uk/research/nl/bea2019st/data/wi+locness_v2.bea19.tar.gz

--2019-04-01 16:50:38--  https://www.cl.cam.ac.uk/research/nl/bea2019st/data/wi+locness_v2.bea19.tar.gz
Resolving www.cl.cam.ac.uk (www.cl.cam.ac.uk)... 128.232.0.20, 2a05:b400:110::80:14
Connecting to www.cl.cam.ac.uk (www.cl.cam.ac.uk)|128.232.0.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4227978 (4.0M) [application/x-gzip]
Saving to: ‘wi+locness_v2.bea19.tar.gz’

wi+locness_v2.bea19 100%[===================>]   4.03M  --.-KB/s    in 0.1s    

2019-04-01 16:50:38 (36.5 MB/s) - ‘wi+locness_v2.bea19.tar.gz’ saved [4227978/4227978]



In [6]:
!tar -xvzf wi+locness_v2.bea19.tar.gz

wi+locness/
wi+locness/json_to_m2.py
wi+locness/licence.wi.txt
wi+locness/readme.txt
wi+locness/license.locness.txt
wi+locness/json/
wi+locness/json/A.dev.json
wi+locness/json/A.train.json
wi+locness/json/B.dev.json
wi+locness/json/B.train.json
wi+locness/json/C.dev.json
wi+locness/json/C.train.json
wi+locness/json/N.dev.json
wi+locness/m2/
wi+locness/m2/ABCN.dev.gold.bea19.m2
wi+locness/m2/A.train.gold.bea19.m2
wi+locness/m2/A.dev.gold.bea19.m2
wi+locness/m2/B.train.gold.bea19.m2
wi+locness/m2/B.dev.gold.bea19.m2
wi+locness/m2/C.train.gold.bea19.m2
wi+locness/m2/C.dev.gold.bea19.m2
wi+locness/m2/N.dev.gold.bea19.m2


In [0]:
#import shutil
#shutil.copy2('content/wi+locness','drive/My Drive/GEC wi+locness/' )                  #doesn't work

In [0]:
m2 = open('wi+locness/m2/A.dev.gold.bea19.m2').read().strip().split('\n\n')[0]

In [9]:
m2.split('\n')

['S It \'s difficult answer at the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .',
 'A 3 3|||M:VERB:FORM|||to|||REQUIRED|||-NONE-|||0',
 'A 4 5|||U:PREP||||||REQUIRED|||-NONE-|||0']

In [0]:
# Apply the edits of a single annotator to generate the corrected sentences.

def corrections(m2_file, out, id=0):
	m2 = open(m2_file).read().strip().split("\n\n")
	out = open(out, "w")
	# Do not apply edits with these error types
	skip = {"noop", "UNK", "Um"}
	
	for sent in m2:
		sent = sent.split("\n")
		cor_sent = sent[0].split()[1:] # Ignore "S "
		edits = sent[1:]
		offset = 0
		for edit in edits:
			edit = edit.split("|||")
			if edit[1] in skip: continue # Ignore certain edits
			coder = int(edit[-1])
			if coder != id: continue # Ignore other coders
			span = edit[0].split()[1:] # Ignore "A "
			start = int(span[0])
			end = int(span[1])
			cor = edit[2].split()
			cor_sent[start+offset:end+offset] = cor
			offset = offset-(end-start)+len(cor)
		out.write(" ".join(cor_sent)+"\n")

In [11]:
root="./wi+locness/m2/"
files=["ABCN.dev.gold.bea19","A.train.gold.bea19","A.dev.gold.bea19","B.train.gold.bea19","B.dev.gold.bea19","C.train.gold.bea19","C.dev.gold.bea19","N.dev.gold.bea19"]

for file in files:
  m2_file=root+file+".m2"
  out_file=root+file+".corrected.txt"
  if os.path.exists(m2_file):
    print("correcting",m2_file)
    corrections(m2_file,out_file)
  else:
    print("error loading",m2_file)

correcting ./wi+locness/m2/ABCN.dev.gold.bea19.m2
correcting ./wi+locness/m2/A.train.gold.bea19.m2
correcting ./wi+locness/m2/A.dev.gold.bea19.m2
correcting ./wi+locness/m2/B.train.gold.bea19.m2
correcting ./wi+locness/m2/B.dev.gold.bea19.m2
correcting ./wi+locness/m2/C.train.gold.bea19.m2
correcting ./wi+locness/m2/C.dev.gold.bea19.m2
correcting ./wi+locness/m2/N.dev.gold.bea19.m2


In [0]:
def in_corr_sentence(m2_file, out_file):
  m2 = open(m2_file).read().strip().split("\n\n")
  out = open(out_file, "w")
  sentences=[]
  for para in m2:
    sent = para.split("\n")[0]
    in_cor_sent = sent.split()[1:] # Ignore "S "
    sentences.append(in_cor_sent)
    out.write(" ".join(in_cor_sent)+"\n")
  
  return sentences

In [13]:
root="./wi+locness/m2/"
files=["ABCN.dev.gold.bea19","A.train.gold.bea19","A.dev.gold.bea19","B.train.gold.bea19","B.dev.gold.bea19","C.train.gold.bea19","C.dev.gold.bea19","N.dev.gold.bea19"]

for file in files:
  m2_file=root+file+".m2"
  out_file=root+file+".incorrect.txt"
  if os.path.exists(m2_file):
    print("generating data",m2_file)
    in_corr_sentence(m2_file,out_file)
  else:
    print("error loading",m2_file)

generating data ./wi+locness/m2/ABCN.dev.gold.bea19.m2
generating data ./wi+locness/m2/A.train.gold.bea19.m2
generating data ./wi+locness/m2/A.dev.gold.bea19.m2
generating data ./wi+locness/m2/B.train.gold.bea19.m2
generating data ./wi+locness/m2/B.dev.gold.bea19.m2
generating data ./wi+locness/m2/C.train.gold.bea19.m2
generating data ./wi+locness/m2/C.dev.gold.bea19.m2
generating data ./wi+locness/m2/N.dev.gold.bea19.m2


# **Generating Train and Dev corpus**


In [14]:
files=["ABCN.dev.gold.bea19.corrected.txt","A.train.gold.bea19.corrected.txt","A.dev.gold.bea19.corrected.txt","B.train.gold.bea19.corrected.txt","B.dev.gold.bea19.corrected.txt","C.train.gold.bea19.corrected.txt","C.dev.gold.bea19.corrected.txt","N.dev.gold.bea19.corrected.txt"]
root="./wi+locness/m2/"
corpus=[]

for file in files:
  with open(root+file) as f:
    l=f.read().strip().split('\n')
    print(root+file,len(l))
    corpus=corpus+l


print("corpus",len(corpus))


./wi+locness/m2/ABCN.dev.gold.bea19.corrected.txt 4384
./wi+locness/m2/A.train.gold.bea19.corrected.txt 10493
./wi+locness/m2/A.dev.gold.bea19.corrected.txt 1037
./wi+locness/m2/B.train.gold.bea19.corrected.txt 13032
./wi+locness/m2/B.dev.gold.bea19.corrected.txt 1290
./wi+locness/m2/C.train.gold.bea19.corrected.txt 10783
./wi+locness/m2/C.dev.gold.bea19.corrected.txt 1069
./wi+locness/m2/N.dev.gold.bea19.corrected.txt 988
corpus 43076


In [15]:
files=["A.train.gold.bea19.corrected.txt","B.train.gold.bea19.corrected.txt","C.train.gold.bea19.corrected.txt"]
root="./wi+locness/m2/"
train_corr_corpus=[]

for file in files:
  with open(root+file) as f:
    l=f.read().strip().split('\n')
    print(root+file,len(l))
    train_corr_corpus=train_corr_corpus+l


print("train_corr_corpus",len(train_corr_corpus))

./wi+locness/m2/A.train.gold.bea19.corrected.txt 10493
./wi+locness/m2/B.train.gold.bea19.corrected.txt 13032
./wi+locness/m2/C.train.gold.bea19.corrected.txt 10783
train_corr_corpus 34308


In [16]:
files=["A.train.gold.bea19.incorrect.txt","B.train.gold.bea19.incorrect.txt","C.train.gold.bea19.incorrect.txt"]
root="./wi+locness/m2/"
train_incorr_corpus=[]

for file in files:
  with open(root+file) as f:
    l=f.read().strip().split('\n')
    print(root+file,len(l))
    train_incorr_corpus=train_incorr_corpus+l


print("train_incorr_corpus",len(train_incorr_corpus))

./wi+locness/m2/A.train.gold.bea19.incorrect.txt 10493
./wi+locness/m2/B.train.gold.bea19.incorrect.txt 13032
./wi+locness/m2/C.train.gold.bea19.incorrect.txt 10783
train_incorr_corpus 34308


In [17]:
files=["A.dev.gold.bea19.corrected.txt","B.dev.gold.bea19.corrected.txt","C.dev.gold.bea19.corrected.txt"]
root="./wi+locness/m2/"
dev_corr_corpus=[]

for file in files:
  with open(root+file) as f:
    l=f.read().strip().split('\n')
    print(root+file,len(l))
    dev_corr_corpus=dev_corr_corpus + l


print("dev_corr_corpus",len(dev_corr_corpus))

./wi+locness/m2/A.dev.gold.bea19.corrected.txt 1037
./wi+locness/m2/B.dev.gold.bea19.corrected.txt 1290
./wi+locness/m2/C.dev.gold.bea19.corrected.txt 1069
dev_corr_corpus 3396


In [18]:
files=["A.dev.gold.bea19.incorrect.txt","B.dev.gold.bea19.incorrect.txt","C.dev.gold.bea19.incorrect.txt"]
root="./wi+locness/m2/"
dev_incorr_corpus=[]

for file in files:
  with open(root+file) as f:
    l=f.read().strip().split('\n')
    print(root+file,len(l))
    dev_incorr_corpus=dev_incorr_corpus + l


print("dev_incorr_corpus",len(dev_incorr_corpus))

./wi+locness/m2/A.dev.gold.bea19.incorrect.txt 1037
./wi+locness/m2/B.dev.gold.bea19.incorrect.txt 1290
./wi+locness/m2/C.dev.gold.bea19.incorrect.txt 1069
dev_incorr_corpus 3396


# **Preprocessing the text**

In [0]:
from keras.preprocessing.text import Tokenizer

tokenizer=Tokenizer(filters='')
tokenizer.fit_on_texts(corpus)

In [0]:
vocabulary=["#unk#","#start#","#end#"]

In [0]:
for word in tokenizer.word_index.keys():
  vocabulary.append(word)

In [0]:
new_tokenizer = Tokenizer(filters='')
new_tokenizer.fit_on_texts(vocabulary)

In [64]:
new_tokenizer.word_index

{'#unk#': 1,
 '#start#': 2,
 '#end#': 3,
 '.': 4,
 ',': 5,
 'the': 6,
 'to': 7,
 'and': 8,
 'a': 9,
 'i': 10,
 'of': 11,
 'in': 12,
 'is': 13,
 'it': 14,
 'that': 15,
 'you': 16,
 'for': 17,
 'my': 18,
 'are': 19,
 'have': 20,
 'be': 21,
 'with': 22,
 'was': 23,
 'we': 24,
 'this': 25,
 'people': 26,
 'as': 27,
 'on': 28,
 'can': 29,
 'they': 30,
 'not': 31,
 'but': 32,
 'will': 33,
 'he': 34,
 'because': 35,
 'do': 36,
 'there': 37,
 'at': 38,
 "'s": 39,
 'so': 40,
 'more': 41,
 'or': 42,
 'all': 43,
 "n't": 44,
 'by': 45,
 'about': 46,
 'if': 47,
 'their': 48,
 'from': 49,
 'like': 50,
 'time': 51,
 'your': 52,
 '-': 53,
 'one': 54,
 'would': 55,
 'when': 56,
 '"': 57,
 'very': 58,
 'me': 59,
 'an': 60,
 'our': 61,
 'has': 62,
 'also': 63,
 'some': 64,
 'had': 65,
 'public': 66,
 'which': 67,
 'transport': 68,
 'am': 69,
 'many': 70,
 'other': 71,
 'them': 72,
 'his': 73,
 'good': 74,
 'think': 75,
 'she': 76,
 'who': 77,
 'could': 78,
 'what': 79,
 'most': 80,
 '?': 81,
 'lot': 82,


In [65]:
print ('tokenizer len = ', len(tokenizer.word_index))
print ('new_tokenizer len = ', len(new_tokenizer.word_index))

tokenizer len =  19598
new_tokenizer len =  19601


In [66]:
vocab_len=len(new_tokenizer.word_index)
print(vocab_len)

19601


In [0]:
from keras.preprocessing.sequence import pad_sequences

In [0]:
vocab = new_tokenizer.word_index

In [0]:

def to_indices_X(sentences):
  indices=[]
  count=0
  for sentence in sentences:
    index=[]
    index.append(vocab["#start#"])
    
    for i in list(nlp(sentence.lower())):
      if i.text in vocab.keys():
        #print(i.text,type(i.text))
        index.append(vocab[i.text])
      else:
        #print("unknown encountered")
        index.append(vocab["#unk#"])
    index.append(vocab["#end#"])
    indices.append(index)
    count=count+1
    print(count, end=" ")
  return indices
    

In [0]:
def to_indices_Y(sentences):
  indices=[]
  count=0
  for sentence in sentences:
    index=[]
    
    for i in list(nlp(sentence.lower())):
      if i.text in vocab.keys():
        #print(i.text,type(i.text))
        index.append(vocab[i.text])
      else:
        #print("unknown encountered")
        index.append(vocab["#unk#"])
    index.append(vocab["#end#"])
    indices.append(index)
    count=count+1
    print(count, end=" ")
  return indices

In [71]:
to_indices_Y(["get every one", "# bro"])

1 2 

[[104, 127, 54, 3], [2650, 10682, 3]]

In [0]:
def find_max_len(corpus):
  max=0
  for sentence in corpus:
    if (max<len(sentence.split())):
      max=len(sentence.split())
  return max
def count(corpus):
  ref=50
  for sentence in corpus:
    if (ref<len(sentence.split())):
      re= ref+1
  return max

In [0]:
from keras.preprocessing.sequence import pad_sequences

def pad_seq(corpus,max_len):
  corpus=pad_sequences(corpus,maxlen=max_len,truncating="post",padding="post")
  return corpus

# **Data Processing and saving**

In [0]:
X_train = pad_seq(to_indices_X(train_incorr_corpus), 46)

In [0]:
Y_train = pad_seq(to_indices_Y(train_corr_corpus), 48)

In [0]:
X_dev = pad_seq(to_indices_X(dev_incorr_corpus), 46)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [0]:
Y_dev = pad_seq(to_indices_Y(dev_corr_corpus), 48)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

In [0]:
np.save('X_train.npy', X_train)

In [0]:
np.save('Y_train.npy', Y_train)

In [0]:
np.save('X_dev.npy', X_dev)

In [0]:
np.save('Y_dev.npy', Y_dev)

In [0]:
Y_train[0]

array([  18,  217,   13,    9, 3526,   53, 7269,  121,   22, 8067, 2000,
       2648,    4,    3,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0], dtype=int32)

# **Embeddings**

In [0]:
def get_embedding_layer(vocab, emb_dim=300):
  
  vocab_len=len(vocab)+1
  
  emb_matrix=np.zeros((vocab_len,emb_dim))
  
  for word,index in vocab.items():
    if word in ["#unk#","#start#","#end#"]:
      pass
    else:
      emb_matrix[index,:]=np.array(nlp(word).vector)
   
  embedding_layer = Embedding(vocab_len,emb_dim)
  embedding_layer.build((None,))
  embedding_layer.set_weights([emb_matrix]) 
  
  return embedding_layer

#**Attention Model**

In [0]:
emb_dim=300                             
Tx=46
Ty=48
input_shape=(Tx,)
n_a=128
n_s=128
out_size=(Ty,len(vocab)+1)

In [0]:
def attention_step(a,s_prev):
  s_prev = repeat(s_prev)
  e = concat([a,s_prev])
  e = dense1(e)
  e = dense2(e)
  alpha = activator(e)
  context = dot([alpha,a])
  
  return context


In [0]:
ost_attention_LSTM=LSTM(n_s,return_state=True,name="post_attention_LSTM")
output_layer=Dense(out_size[1],activation="softmax",name="output_dense")
embedding_layer=get_embedding_layer(vocab)

# Seq2Seq **Model**

In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Bidirectional, LSTM, Dense, Dot, RepeatVector, Concatenate, Permute, Activation, Lambda, Input,Embedding,SpatialDropout1D

In [0]:
Tx = 46
Ty = 48
latent_dim=128
num_decoder_tokens = len(vocab)+1

In [0]:

encoder_inputs = Input(shape=(Tx,))
embedding_enc1 = get_embedding_layer(vocab)
embedding_enc2= embedding_enc1(encoder_inputs)
embedding_enc3 = SpatialDropout1D(0.25)(embedding_enc2)
  
encoder = Bidirectional(LSTM(latent_dim, return_state=True))
lstm, forward_h, forward_c, backward_h, backward_c= encoder(embedding_enc3) 
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])
encoder_states = [state_h, state_c]              #use only hidden states of the encoder

decoder_inputs = Input(shape=(None,))               # none has been done as while training, Ty=48 and while sampling Tx=46
embedding_dec1 = get_embedding_layer(vocab)
embedding_dec2 = embedding_dec1(decoder_inputs)
embedding_dec3 = SpatialDropout1D(0.25)(embedding_dec2)                            #this takes time to load .....
      

decoder_lstm = LSTM(2*latent_dim, return_sequences=True, return_state=True,)
decoder_outputs, _, _ = decoder_lstm(embedding_dec3, initial_state=encoder_states)
  
decoder_dense = Dense(num_decoder_tokens, activation='softmax') 
decoder_outputs = decoder_dense(decoder_outputs)
  

  

In [0]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.load_weights('drive/My Drive/Copy of 01-04-model-ep004-loss0.421-val_loss1.184.h5')

# Compile

In [0]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [81]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 46)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 46, 300)      5880600     input_5[0][0]                    
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 46, 300)      0           embedding_3[0][0]                
__________________________________________________________________________________________________
embedding_

In [0]:
from keras.utils import plot_model
plot_model(model, to_file='model.png',show_shapes=True)

In [0]:
train_encoder_X = np.load('train_encoder_in.npy')
train_decoder_X = np.load('train_decoder_in.npy')
train_decoder_Y = np.load('train_decoder_out.npy')
dev_encoder_X = np.load('dev_encoder_in.npy')
dev_decoder_X = np.load('dev_decoder_in.npy')
dev_decoder_Y = np.load('dev_decoder_out.npy')


In [83]:
train_decoder_X.shape

(34308, 48)

# **Generators**

In [0]:
from keras.utils import to_categorical
from keras.utils import Sequence

In [0]:
class Generator(Sequence):
  def __init__(self, enc_inp, dec_inp, dec_out, batch=6):
    self.enc_inp=enc_inp
    self.dec_inp=dec_inp
    self.dec_out=dec_out
    self.batch=batch
    
    
  def __len__(self):
    return int(np.floor(len(self.enc_inp) / self.batch))
  
  def __getitem__(self, index):
    enc_inp_batch=self.enc_inp[index*self.batch:(index+1)*self.batch]
    dec_inp_batch=self.dec_inp[index*self.batch:(index+1)*self.batch]
    dec_out_batch=self.dec_out[index*self.batch:(index+1)*self.batch]

    dec_out_batch=to_categorical(dec_out_batch,num_classes=len(vocab)+1)
    
    return [np.array(enc_inp_batch),np.array(dec_inp_batch)], np.array(dec_out_batch)

In [0]:
training_gen=Generator(train_encoder_X, train_decoder_X, train_decoder_Y, 32)
val_gen=Generator(dev_encoder_X, dev_decoder_X, dev_decoder_Y, 32)

# Training

In [0]:
from keras.callbacks import ModelCheckpoint , ReduceLROnPlateau
filepath = 'drive/My Drive/weights/GEC/01-04-model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'

model_checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


In [0]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [0]:
history=model.fit_generator(generator=training_gen, 
                    use_multiprocessing=True,
                    verbose=1,
                    validation_data=val_gen,
                    callbacks= [model_checkpoint,learning_rate_reduction],
                    initial_epoch=0,
                    epochs=5)

Epoch 1/5
1072/1072 [==============================] - 581s 542ms/step - loss: 0.4995 - acc: 0.8827 - val_loss: 1.1845 - val_acc: 0.8074

Epoch 00001: val_loss improved from inf to 1.18453, saving model to drive/My Drive/weights/GEC/01-04-model-ep001-loss0.499-val_loss1.185.h5


/usr/local/lib/python3.6/dist-packages/keras/engine/network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'concatenate_1/concat:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'concatenate_2/concat:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/5
1072/1072 [==============================] - 584s 545ms/step - loss: 0.4786 - acc: 0.8868 - val_loss: 1.1866 - val_acc: 0.8082

Epoch 00002: val_loss did not improve from 1.18453
Epoch 3/5
1072/1072 [==============================] - 583s 544ms/step - loss: 0.4611 - acc: 0.8901 - val_loss: 1.1909 - val_acc: 0.8078

Epoch 00003: val_loss did not improve from 1.18453

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 4/5
1072/1072 [==============================] - 582s 542ms/step - loss: 0.4214 - acc: 0.8990 - val_loss: 1.1838 - val_acc: 0.8093

Epoch 00004: val_loss improved from 1.18453 to 1.18382, saving model to drive/My Drive/weights/GEC/01-04-model-ep004-loss0.421-val_loss1.184.h5
Epoch 5/5
1072/1072 [==============================] - 582s 543ms/step - loss: 0.4070 - acc: 0.9021 - val_loss: 1.1851 - val_acc: 0.8095

Epoch 00005: val_loss did not improve from 1.18382


#**Sampling**

---


In [84]:
###### encoder model #######
encoder_model=Model(inputs=encoder_inputs,outputs=encoder_states)
encoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 46)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 46, 300)      5880600     input_5[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_3 (SpatialDro (None, 46, 300)      0           embedding_3[0][0]                
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) [(None, 256), (None, 439296      spatial_dropout1d_3[0][0]        
__________________________________________________________________________________________________
concatenat

In [85]:
input_h=Input(shape=(256,))
input_c=Input(shape=(256,))

decoder_input_states=[input_h,input_c]

dec_emb=embedding_dec1(decoder_inputs)
decoder_outputs2,decoder_state_h2,decoder_state_c2=decoder_lstm(dec_emb,initial_state=decoder_input_states)
decoder_states2 = [decoder_state_h2, decoder_state_c2]
decoder_outputs2=decoder_dense(decoder_outputs2)

decoder_model=Model(inputs=[decoder_inputs]+decoder_input_states,outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 300)    5880600     input_6[0][0]                    
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
lstm_4 (LS

In [0]:
!ls

In [0]:
def predict_sentence(inp_sequence):
  
  
  #print(inp_sequence.shape)
  dec_states=encoder_model.predict(inp_sequence)
  target_seq=np.zeros((1,1))
  target_seq[0,0]=vocab["#start#"]
  
  
  sentence=[]
  
  while True:
    
    output_tokens,h,c=decoder_model.predict( [target_seq] + dec_states )
    
    
    index=np.argmax(output_tokens[0,-1,:])
    word=new_tokenizer.index_word[index]
    
    sentence.append(word)
    
    if word=="#end#" or len(sentence)>40:
      break
      
    target_seq[0,0]=index
    dec_states=[h,c]
  
  #print(output_tokens.shape)
  return sentence



In [0]:
i=4
seq=pad_seq(to_indices_X(train_incorr_corpus[i:i+1]),46)
train_incorr_corpus[i:i+1]

In [88]:
for i in [3843]:
  seq=pad_seq(to_indices_X(train_incorr_corpus[i:i+1]),46)
  #print([index_to_word[word] for word in seq[0] if word!=0])
  #print(seq)
  #seq=seq[0]
  #print(seq)
  sentence=predict_sentence(seq)
  #print(sentence)
  
  inc_sentence=[new_tokenizer.index_word[j] for j in seq[0] if j!=0]
  corr_sentence=[j for j in sentence]
  
  print()
  print(train_incorr_corpus[i:i+1])
  print("correct sentence:"," ".join(corr_sentence))

1 
["Particulary I would like to point Mathew McConnahy 's role ."]
correct sentence: in particular , i would like to point out mathew mcconnahy 's role . #end#


In [59]:
train_incorr_corpus[i:i+1]

["Particulary I would like to point Mathew McConnahy 's role ."]